# Amazon Bedrockモデルをコード生成に使用する

> *このノートブックは、SageMaker Studioの **`Data Science 3.0`** カーネルで実行してください*

## 概要

このノートブックでは、テキストプロンプトに基づいてLLMを使用してコードを生成する方法を示します。

今回は Amazon Bedrock の Claude を Boto3 API で使用します。

この例で使用されているプロンプトは、プロンプト以外のテキストの例を提供していないため、ゼロショットプロンプトと呼ばれます。

**注:** *このノートブックは、AWS環境の内外で実行できます。*

#### コンテキスト
Amazon Bedrock のコード生成機能をデモするために、Boto3 クライアントを使用して Amazon Bedrock API と通信する方法を検証します。さまざまな設定や、シンプルな入力で、望ましい出力につながる方法を探索します。

#### パターン
Amazon Bedrock API に、タスク、指示、およびモデルへのインプットからなるシンプルなインプットを提供します。ここでの目的は、強力な LLM が与えられたタスクを理解し、有効な出力を生成できることを確認することです。

![](./images/bedrock-code-gen.png)

#### ユースケース
Amazon Bedrock のモデルの機能を確認するために、コード生成のユースケースを取り上げましょう。

#### ペルソナ
あなたは AnyCompany 社のデータアナリストです。会社は、過去1年間のさまざまな製品の販売実績を理解したいと考えています。sales.csv というデータセットが提供されています。データセットには次の列が含まれています:

- 日付 (YYYY-MM-DD 形式) 
- Product_ID (各製品に一意の識別子)
- Price (各製品の販売価格)

#### 実装
このユースケースを満たすために、このノートブックでは、指定されたプロンプトに対してコードを生成する方法を示します。 Amazon Bedrock API と Boto3 クライアントを使用して、Anthropic Claude v2 を使用します。

In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from labutils import bedrock, print_ww


# ---- ⚠️ AWS 環境の設定に応じて、以下の行のコメントを外して編集してください。 ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

## コード生成

上記で説明したユースケースを想定して Python プログラムを生成するための Amazon Bedrock サービスの入力を準備しましょう。

#### ラボセットアップ - ラボ用にサンプルの sales.csv ファイルを作成する



In [ ]:
# create sales.csv file
import csv

data = [
    ["date", "product_id", "price", "units_sold"],
    ["2023-01-01", "P001", 50, 20],
    ["2023-01-02", "P002", 60, 15],
    ["2023-01-03", "P001", 50, 18],
    ["2023-01-04", "P003", 70, 30],
    ["2023-01-05", "P001", 50, 25],
    ["2023-01-06", "P002", 60, 22],
    ["2023-01-07", "P003", 70, 24],
    ["2023-01-08", "P001", 50, 28],
    ["2023-01-09", "P002", 60, 17],
    ["2023-01-10", "P003", 70, 29],
    ["2023-02-11", "P001", 50, 23],
    ["2023-02-12", "P002", 60, 19],
    ["2023-02-13", "P001", 50, 21],
    ["2023-02-14", "P003", 70, 31],
    ["2023-03-15", "P001", 50, 26],
    ["2023-03-16", "P002", 60, 20],
    ["2023-03-17", "P003", 70, 33],
    ["2023-04-18", "P001", 50, 27],
    ["2023-04-19", "P002", 60, 18],
    ["2023-04-20", "P003", 70, 32],
    ["2023-04-21", "P001", 50, 22],
    ["2023-04-22", "P002", 60, 16],
    ["2023-04-23", "P003", 70, 34],
    ["2023-05-24", "P001", 50, 24],
    ["2023-05-25", "P002", 60, 21]
]

# Write data to sales.csv
with open('sales.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(data)

print("sales.csv has been created!")

#### Amazon Bedrock で生成した Python プログラムで売上を分析する

In [ ]:
# Create the prompt
# Analyzing sales

prompt_data = """

Human: You have a CSV, sales.csv, with columns:
- date (YYYY-MM-DD)
- product_id
- price
- units_sold

Create a python program to analyze the sales data from a CSV file. The program should be able to read the data, and determine below:

- Total revenue for the year
- The product with the highest revenue
- The date with the highest revenue
- Visualize monthly sales using a bar chart

Ensure the code is syntactically correct, bug-free, optimized, not span multiple lines unnessarily, and prefer to use standard libraries. Return only python code without any surrounding text, explanation or context.

Assistant:
"""

Anthropic Claude V2 モデルを使用して始めましょう。

In [ ]:
# Claude - Body Syntex
body = json.dumps({
                    "prompt": prompt_data,
                    "max_tokens_to_sample":4096,
                    "temperature":0.5,
                    "top_k":250,
                    "top_p":0.5,
                    "stop_sequences": ["\n\nHuman:"]
                  }) 

#### Anthropic Claude v2 モデルを実行する

In [ ]:
modelId = 'anthropic.claude-v2:1' # 使用する環境に合わせて、適切なモデルバージョンを指定してください。
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print_ww(response_body.get('completion'))

#### (オプション) 検証のために Bedrock で生成されたコードを実行してください。プリントされた出力が切り捨てられる可能性があるため、テキストエディタに移動して生成されたコードをコピーしてください。以下のセルのコードを置き換えます。

In [ ]:
# Sample Generated Python Code ( Generated with Amazon Bedrock in previous step)

import csv
from collections import defaultdict
import matplotlib.pyplot as plt

revenue = 0
monthly_revenue = defaultdict(int)
product_revenue = defaultdict(int)
max_revenue = 0
max_revenue_date = ''
max_revenue_product = ''

with open('sales.csv') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        date = row[0]
        product = row[1]
        price = float(row[2])
        units = int(row[3])

        revenue += price * units
        product_revenue[product] += price * units
        monthly_revenue[date[:7]] += price * units

        if revenue > max_revenue:
            max_revenue = revenue
            max_revenue_date = date
            max_revenue_product = product

months = list(monthly_revenue.keys())
values = list(monthly_revenue.values())

plt.bar(months, values)
plt.xlabel('Month')
plt.ylabel('Revenue')
plt.title('Monthly Revenue')
plt.show()

print('Total Revenue:', revenue)
print('Product with max revenue:', max_revenue_product)
print('Date with max revenue:', max_revenue_date)

## まとめ
`Boto3` SDK を使用して、Amazon Bedrock API へのシンプルな接続方法を確認しました。この API を使用して、指定された販売データを分析および可視化する Python プログラムを生成できました。

### ポイント
- このノートブックを調整して、Amazon Titan や AI21 Labs Jurassic など、Amazon Bedrock を通じて利用できるさまざまなモデルを実験してください。

## Thank You